In [119]:
import numpy as np
from matplotlib import pyplot as plt
import padasip as pa
from scipy import fftpack
import processing_signals as ps

# plt.figure(figsize=(10, 7))

In [120]:
regexp = r'[+-]?\d+\.?\d*'
BP1 = np.fromregex('BP1.lcn', regexp, [('num', np.float64)])['num']
BP2 = np.fromregex('BP2.lcn', regexp, [('num', np.float64)])['num']
OPERATOR = np.fromregex('OPERATOR.lcn', regexp, [('num', np.float64)])['num']

print(BP1.size, BP2.size, OPERATOR.size)

501 501 1025


Моделируем входной и выходной 2d сигнал на адаптивный фильтр

In [ ]:
plt.figure(figsize=(15, 6))

plt.subplot(311)
plt.plot(BP1)
plt.title("BP1")

plt.subplot(312)
plt.plot(BP2)
plt.title("BP2")

plt.subplot(313)
plt.plot(OPERATOR)
plt.title("OPERATOR")
plt.show()

In [ ]:
approx_BP2 = ps.processing_of_signal(BP1, OPERATOR[::])

plt.figure(figsize=(15, 6))

# plt.plot(BP2)
plt.plot(approx_BP2)
plt.title("approx_BP2")

plt.show()

In [ ]:
approx_weights_nlms = ps.fitting_of_weights(BP1, BP2, 10, 0.5, 0.000).flatten()
print(approx_weights_nlms)

In [ ]:
approx_weights_rls = ps.RLS(BP1, BP2, 10, 0.9).flatten()
print(approx_weights_rls)

In [ ]:
approx_BP2_nlms = ps.processing_of_signal(BP1, approx_weights_nlms)
approx_BP2_rls = ps.processing_of_signal(BP1, approx_weights_rls)

plt.figure(figsize=(15, 6))

plt.subplot(211)
plt.plot(BP2)
plt.plot(approx_BP2_nlms)
plt.title("nlms")

plt.subplot(212)
plt.plot(BP2)
plt.plot(approx_BP2_rls)
plt.title("rls")

plt.show()

# Padasip

In [ ]:
number_of_weights_p_nlms = 10
number_of_weights_p_rls = 10

In [ ]:
first_input_1d_nlms = pa.input_from_history(BP1, number_of_weights_p_nlms)
first_input_1d_rls = pa.input_from_history(BP1, number_of_weights_p_rls)
second_input_1d = BP2

In [ ]:
f = pa.filters.FilterNLMS(n=number_of_weights_p_nlms, mu=2*0.5, eps=0, w="zeros")
y, e, w = f.run(second_input_1d[:-(number_of_weights_p_nlms-1)], first_input_1d_nlms)

approx_weights_p_nlms = w[-1]
print(approx_weights_p_nlms)

In [ ]:
f = pa.filters.FilterRLS(n=number_of_weights_p_rls, mu=0.9, w="zeros")
y, e, w = f.run(second_input_1d[:-(number_of_weights_p_rls-1)], first_input_1d_rls)

approx_weights_p_rls = w[-1]
print(approx_weights_p_rls)

In [ ]:
approx_BP2_p_nlms = ps.processing_of_signal(BP1, approx_weights_p_nlms)
approx_BP2_p_rls = ps.processing_of_signal(BP1, approx_weights_p_rls)

plt.figure(figsize=(15, 6))

plt.subplot(211)
plt.plot(BP2)
plt.plot(approx_BP2_p_nlms)
plt.title("nlms")

plt.subplot(212)
plt.plot(BP2)
plt.plot(approx_BP2_p_rls)
plt.title("rls")

plt.show()

In [ ]:
# print(f"MSE = {ps.MSE(BP2, approx_BP2):.3e}")
# print(f"ME = {ps.ME(BP2, approx_BP2):.3e}")
print("My NLMS")
print(f"MSE = {ps.MSE(BP2, approx_BP2_nlms):.3e}")
print(f"ME = {ps.ME(BP2, approx_BP2_nlms):.3e}")
print("Padasip NLMS")
print(f"MSE = {ps.MSE(BP2, approx_BP2_p_nlms):.3e}")
print(f"ME = {ps.ME(BP2, approx_BP2_p_nlms):.3e}")
print("My RLS")
print(f"MSE = {ps.MSE(BP2, approx_BP2_rls):.3e}")
print(f"ME = {ps.ME(BP2,approx_BP2_rls):.3e}")
print("Padasip RLS")
print(f"MSE = {ps.MSE(BP2, approx_BP2_p_rls):.3e}")
print(f"ME = {ps.ME(BP2, approx_BP2_p_rls):.3e}")